- [참고 data2vec audio:사용 안하기로함]: https://towhee.io/audio-embedding/data2vec
- [참고 data2vec text]: https://huggingface.co/docs/datasets/nlp_load
- [참고 data2vec load audio*]:https://huggingface.co/docs/datasets/audio_load
- [참고 data2vec embedding]: https://huggingface.co/docs/transformers/model_doc/data2vec#transformers.Data2VecAudioModel
- [참고 handling pytorch dataset]:https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel
- [참고 add data to dataset]: https://pypi.org/project/datasets/

In [1]:
import torch
from transformers import Wav2Vec2Processor, Data2VecAudioModel, AutoFeatureExtractor
from datasets import load_dataset, Dataset, Audio, Features
from scipy.io import wavfile # to be added
from glob import glob

processor = Wav2Vec2Processor.from_pretrained("facebook/data2vec-audio-base-960h")
model = Data2VecAudioModel.from_pretrained("facebook/data2vec-audio-base-960h")


/home/arplab/project/lou/multi_modal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at facebook/data2vec-audio-base-960h were not used when initializing Data2VecAudioModel: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecAudioModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Audio Embedding

In [2]:
# from towhee import pipe, ops, DataCollection # to be added
# p = (pipe.input('path')
#         .map('path', 'frame', ops.audio_decode.ffmpeg(sample_rate=16000))
#         .map('frame', 'vecs', ops.audio_embedding.data2vec(model_name='facebook/data2vec-audio-base-960h'))
#         .output('path', 'vecs'))

# DataCollection(p('./org_KEMDy20/Session01/Sess01_script01_User001F_001.wav')).show()

In [3]:
wav_file_lst = glob('./org_KEMDy20/Session01/*.wav')
wav_file_lst[:3]

['./org_KEMDy20/Session01/Sess01_script03_User002M_010.wav',
 './org_KEMDy20/Session01/Sess01_script03_User001F_014.wav',
 './org_KEMDy20/Session01/Sess01_script05_User002M_001.wav']

In [4]:
import datasets

dataset = Dataset.from_dict({"wav": wav_file_lst }).cast_column("wav", Audio())
dataset[0]["wav"]

{'path': './org_KEMDy20/Session01/Sess01_script03_User002M_010.wav',
 'array': array([-0.00018311,  0.00057983,  0.00076294, ...,  0.00259399,
         0.00115967,  0.00036621], dtype=float32),
 'sampling_rate': 16000}

- dataset에 데이터 더하는 방법 찾아보기

In [5]:
dataset

Dataset({
    features: ['wav'],
    num_rows: 311
})

In [6]:
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/data2vec-audio-base-960h")

In [7]:
# audio file is decoded on the fly
input_values = []
for i in range(len(dataset)):
    input = feature_extractor(dataset[i]["wav"]["array"], sampling_rate=dataset[0]['wav']['sampling_rate'], return_attention_mask=False)
    input_values.append(input['input_values'][0])
    

input_values_feature = Features({'embedded_wav': input_values})
input_values_feature['embedded_wav'][:3]

2023-03-26 20:34:06.452296: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-26 20:34:07.664057: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11/include:/usr/local/cuda-11/lib64::/usr/local/cuda/extras/CUPTI/lib64
2023-03-26 20:34:07.664177: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11/include:/usr/local/cuda

[array([-0.00228672,  0.01021109,  0.01321056, ...,  0.04320529,
         0.01970942,  0.0067117 ], dtype=float32),
 array([-0.13987999, -0.1284736 , -0.1221367 , ..., -0.01567697,
        -0.01567697, -0.02454861], dtype=float32),
 array([ 0.6008278 ,  0.6016405 ,  0.61383116, ..., -0.02495759,
        -0.02983384, -0.01926864], dtype=float32)]

In [8]:
dataset_new = dataset.add_column(name = "embedded_wav", column = input_values_feature['embedded_wav'])
dataset_new

Dataset({
    features: ['wav', 'embedded_wav'],
    num_rows: 311
})

In [9]:
len(dataset_new['embedded_wav'])


311

## Text Embedding


- 참고: https://huggingface.co/docs/datasets/nlp_load
- 참고: https://huggingface.co/docs/transformers/v4.27.2/en/model_doc/data2vec#transformers.Data2VecTextModel
- 참고 한국어 pretrained model: https://huggingface.co/Junmai/KR-Data2VecText-v1

In [10]:
from transformers import PreTrainedTokenizerFast, BartModel
from transformers import AutoModel, AutoTokenizer
import pandas as pd



In [11]:
# text data loading
txt_file_lst = glob('./org_KEMDy20/Session01/*.txt')
txt_file_lst[:3]

['./org_KEMDy20/Session01/Sess01_script06_User002M_009.txt',
 './org_KEMDy20/Session01/Sess01_script01_User001F_007.txt',
 './org_KEMDy20/Session01/Sess01_script01_User001F_001.txt']

In [12]:
# 참고: https://huggingface.co/gogamza/kobart-base-v2
# 참고: https://github.com/BM-K/Sentence-Embedding-Is-All-You-Need

# model = AutoModel.from_pretrained('gogamza/kobart-base-v2')  # or 'BM-K/KoSimCSE-bert-multitask'
# tokenizer = AutoTokenizer.from_pretrained('gogamza/kobart-base-v2')  # or 'BM-K/KoSimCSE-bert-multitask'
checkpoint = 'BM-K/KoSimCSE-bert-multitask'
model = AutoModel.from_pretrained(checkpoint)  # or 'BM-K/KoSimCSE-bert-multitask'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)  # or 'BM-K/KoSimCSE-bert-multitask'


In [13]:
import re
sentences = []

for i in txt_file_lst:    
    f = open(i, 'r')
    line = f.readline()
    line = re.sub('l/\n|c/|b/|n/|l/|u/', '', line)
    line = re.sub('  ', ' ', line)
    line = line.rstrip().lstrip()
    sentences.append(line)
    f.close()
    
    
sentences[21:40]

['그것 그것 땜에 당 그거 땜에 당황스러웠던 적은 없었어?',
 '아니면 어 어.',
 '그니까 그런 경험이 음 솔직히 거의 없는 거 같은데 약간 명절에 시골에 내려 가면은 약간 큰 아빠?',
 '그 가영이는 뭐 주변에서 막 이렇게 보고 가영이가 그거 했던 적은 웃겼던 건 적은 없었어?',
 '근데 내가 만약에 열두시에 연락을 해서 어 저 한시간 정도 더 놀다 갈게요 했는데 그 한시까지 안오잖아?',
 '이제 그거에 대해서는 어린 나이에도 대표를 다시는 여성분들도 계시는데 굳이 그거를 비율을 맞춰야 되느냐에 대해서 조금 그런 거 같애.',
 '근데 그거는 굳이 약간 남녀 그런 문제가 아니라 누가 도우려고 하든 간에 내가 도움을 받는 걸 싫어해서 그런 거야.',
 '어 좀 비슷하게 라고는 말하기 좀 그런데 약간 버스를 타다가 넘어 그니까 되게 급 정차를 해 하는 바람에 버스가 그래서 막.',
 '내가 혼잣말하거나 전화를 막 하고 있는데 집에서 자기 혼자 네 부르셨나요 막 이러거나 되게 띠링 소리 나다가 자기 혼자 갑자기 노래를 트는 경우가 있어.',
 '글쎄 일단은 나는 그 말이 살짝 좀 이해가 안 됐어, 약간.',
 '같은 아파트 내에 있는 어른들이나 그런 사람들한텐 다 인사하고 다녔어.',
 '말도 말도 안 했는데 그냥 막 떠.',
 '혹시 여기 무슨 검사 받았 적 있었냐고 나한테 검사받았던 환자시냐고 묻는 거야.',
 '그 꼰대에 특징을 갖고 있다거나 막 그런 거?',
 '그런 면에서 되게 아버지가 뭐라 하시거나?',
 '한 여섯시까지는 들어가야 된다라는 게 있었는데 그거 그 규칙이 불문율이라고 했어야 되나 근데 그거를 오빠가 깨버렸거든.',
 '어 최근에 어디를 가면?',
 '물론 같이 공부를 했을 때 똑같은 사람들도 있겠지만.',
 '우리 집은 되게 자유로워 가지고 솔직히 아빠가 이게 어때라고 하면 본인 생 그니까 가족들 생각이랑 안 맞으면 그냥 자기 주관대로 가거든, 다들.']

In [15]:
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
embeddings, _ = model(**inputs, return_dict=False)

In [26]:
embeddings

tensor([[[-2.3365e-01, -1.0966e+00,  6.5578e-01,  ...,  6.1874e-01,
           1.2647e-01,  5.4168e-01],
         [-1.2150e+00, -2.3270e-01,  5.2826e-01,  ..., -1.0512e-02,
           8.8822e-01, -3.0252e-01],
         [-2.2442e-01, -7.0670e-01,  9.5867e-01,  ...,  4.1815e-01,
           4.2847e-01, -1.8841e-01],
         ...,
         [-2.0204e-01, -2.0759e-01,  7.5531e-01,  ...,  1.4437e-01,
           1.1744e+00,  3.6902e-01],
         [-2.5445e-01, -9.7568e-01,  5.1529e-01,  ...,  2.7664e-01,
           7.4539e-01,  1.7495e-01],
         [-1.0302e-01, -9.4838e-01,  3.5005e-01,  ...,  6.1108e-01,
           1.0481e+00,  2.5768e-01]],

        [[ 5.5688e-01, -5.6695e-01, -2.1437e-01,  ...,  1.6343e-01,
          -3.6775e-01,  3.1689e+00],
         [ 8.2378e-01, -1.3360e+00,  2.6778e-01,  ..., -3.1469e-01,
          -4.5207e-01,  2.9279e+00],
         [ 5.7025e-01, -5.9719e-01,  2.4914e-02,  ..., -4.1381e-01,
          -4.7057e-01,  2.3544e+00],
         ...,
         [ 8.7290e-01, -1

In [27]:
print(len(embeddings))
dataset_new

311


Dataset({
    features: ['wav', 'embedded_wav'],
    num_rows: 311
})

In [32]:
dataset_new

Dataset({
    features: ['wav', 'embedded_wav'],
    num_rows: 311
})

In [34]:
# dataset_new = dataset_new.add_column(name = "embedded_text", column = embeddings)
# dataset_new

ArrowInvalid: Could not convert tensor([[-0.2337, -1.0966,  0.6558,  ...,  0.6187,  0.1265,  0.5417],
        [-1.2150, -0.2327,  0.5283,  ..., -0.0105,  0.8882, -0.3025],
        [-0.2244, -0.7067,  0.9587,  ...,  0.4182,  0.4285, -0.1884],
        ...,
        [-0.2020, -0.2076,  0.7553,  ...,  0.1444,  1.1744,  0.3690],
        [-0.2544, -0.9757,  0.5153,  ...,  0.2766,  0.7454,  0.1749],
        [-0.1030, -0.9484,  0.3500,  ...,  0.6111,  1.0481,  0.2577]],
       grad_fn=<UnbindBackward0>) with type Tensor: did not recognize Python value type when inferring an Arrow data type